In [175]:
import pandas as pd
from tqdm.notebook import tqdm
import numpy as np

In [176]:
df = pd.read_csv('data/train_adj.csv')

In [177]:
df.ts = pd.to_datetime(df.ts, format = "%Y-%m-%d %H:%M:%S")

In [178]:
sensing_mat = df.sensing_material.unique()
fixed_cols = ['T', 'RH', 'P', 'sin_hour', 'cos_hour', 'sin_ordate','cos_ordate', 'sin_weekday', 'cos_weekday']
sens_unfo = ['{}_res', '{}_heat_r', '{}_heat_V','{}_age']
input_cols = ['signal_res', 'heater_res', 'heater_volt', 'age']

sens_cols = [item.format(sens) for sens in [it for se in sensing_mat for it in [se+'-1',se+'-2']] for item in sens_unfo] + ['ts']
out_cols = sens_cols + fixed_cols
out_ds = pd.DataFrame(columns=out_cols)

In [179]:
df_red = df.drop_duplicates(subset=['ts'])
df_red.index = np.arange(1, len(df_red)+1)
for col in fixed_cols:
    out_ds[col] = df_red[col]
out_ds.ts = df_red.ts

In [181]:
for out_i, (hour, group_hour) in tqdm(enumerate(df.groupby('ts')), total=len(out_ds)):
    for sens_mat, sens_df in group_hour.groupby('sensing_material'):
        for i, sens_line in enumerate(sens_df.iterrows()):
            for col, inpu in zip(sens_unfo, input_cols):
                out_ds.at[out_i, col.format(sens_mat + '-' + str(i+1))] = sens_line[1][inpu]

  0%|          | 0/12199 [00:00<?, ?it/s]

In [182]:
out_ds

,ZnOR-1_res,ZnOR-1_heat_r,ZnOR-1_heat_V,ZnOR-1_age,ZnOR-2_res,ZnOR-2_heat_r,ZnOR-2_heat_V,ZnOR-2_age,LaFeO3-1_res,LaFeO3-1_heat_r,...,ts,T,RH,P,sin_hour,cos_hour,sin_ordate,cos_ordate,sin_weekday,cos_weekday
1,122969.326923,100.6,3.954423,0,159414.288462,98.575,3.935385,0,10308039.057692,87.063462,...,2021-01-13 08:00:00,26.691667,22.616667,991.958333,0.259013,-0.038092,-0.000596,0.017157,0.816183,-0.373533
2,166768.85,101.523333,3.996333,0,172682.866667,99.485,3.981,0,610747.866667,87.981667,...,2021-01-13 09:00:00,40.938462,11.401923,991.523077,0.107892,-0.238533,-0.000596,0.017157,0.816183,-0.373533
3,178354.816667,101.588333,3.995833,0,169701.516667,99.515,3.981667,0,598889.35,88.031667,...,2021-01-13 10:00:00,46.071667,5.108333,990.721667,-0.142424,-0.219668,-0.000596,0.017157,0.816183,-0.373533
4,614298.333333,101.188333,3.957167,0,165691.266667,99.566667,3.981833,0,596706.5,88.088333,...,2021-01-13 11:00:00,46.831667,4.700000,990.098333,-0.261797,0.001159,-0.000596,0.017157,0.816183,-0.373533
5,196725.583333,102.171667,4.0395,0,154136.25,99.595,3.982333,0,596870.883333,88.161667,...,2021-01-13 12:00:00,47.765000,4.463333,989.515000,-0.140474,0.220920,-0.000596,0.017157,0.816183,-0.373533
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12196,499000000.0,126.755,3.890667,540,499000000.0,125.365,3.897,540,947867.6,99.516667,...,2022-07-07 20:00:00,36.333898,29.774576,991.242373,0.239009,0.106836,-0.006505,0.015887,0.126669,-0.888615
12197,499000000.0,126.811667,3.891167,540,499000000.0,125.358333,3.896667,540,941445.733333,99.513333,...,2022-07-07 21:00:00,36.095000,30.428333,992.555000,0.219036,-0.143395,-0.006505,0.015887,0.126669,-0.888615
12198,499000000.0,126.821667,3.891167,540,499000000.0,125.353333,3.8965,540,900261.7,99.535,...,2022-07-07 22:00:00,36.016667,29.701667,993.073333,-0.002317,-0.261789,-0.006505,0.015887,0.126669,-0.888615
12199,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2022-07-07 23:00:00,35.941667,29.576667,993.590000,-0.221540,-0.139495,-0.006505,0.015887,0.126669,-0.888615


In [184]:
out_ds.to_csv('data/parallel_appa1.csv')

In [185]:
sensing_mat

array(['ZnOR', 'LaFeO3', 'WO3', 'ZnO', 'ZnOg', 'STN', 'SmFeO3'],
      dtype=object)